<center><h1>VC10: Algoritmo APRIORI</h1></center>

# RECUERDA RELLENAR TUS DATOS A CONTINUACIÓN ANTES DE HACER NADA

In [5]:
# ===============================================================#
# Rellena AQUÍ tu nombre y apellidos antes de hacer nada
# ===============================================================#

NOMBRE = 'CarlosEsteban'
APELLIDOS = 'PosadaMejia'

# ===============================================================#
# NO MODIFIQUES ESTA PORCIÓN DE CÓDIGO, ES PARA LA EVALUACIÓN
# MUY IMPORTANTE: NO MODIFICAR
# ===============================================================#
from matplotlib.backends.backend_agg import FigureCanvasAgg
def encode_figure(fig):
    canvas = FigureCanvasAgg(fig)
    canvas.draw()
    img, (width, height) = canvas.print_to_buffer()
    return {'img': img, 'width': width, 'height': height}

answers = {}
answers['name'] = NOMBRE
answers['surname'] = APELLIDOS
answers['subject'] = '06MAIR_10_A_2020-21_ANS'
answers['ex'] = 'N4.2'
# ===============================================================#

El algoritmo Apriori es un procedimiento para encontrar subsets frecuentes de ítems. En el caso de la cesta de la compra serían conjuntos de productos que suelen comprarse simultáneamente. 


Se podría decir que el algoritmo Apriori es una búsqueda en anchura donde, en primer lugar, se buscan todos los subconjuntos $X$ de tamaño 1 que tienen un mínimo soporte sobre el conjunto de transacciones $S$, $soporte(X;S)\geq minS$, donde el soporte es una métrica que se define como:
$$soporte(X;S)=\frac{|\{T\in S:X\subseteq T\}}{|S|}$$

Así, la primera tarea consiste en detectar todos los subconjuntos de tamaño $1$:

In [6]:
import numpy as np

def generarC1(S):
    C1 = []
    for transaccion in S:
        for item in transaccion:
            if [item] not in C1:
                C1.append([item])
                
    C1.sort()
    return list(map(frozenset, C1)) # usando un `frozenset´ podemos usarlo como una `key´ de un diccionario


Hagamos una prueba con este pequeño conjunto de transacciones:


In [7]:
transacciones = np.array([[2, 3, 4], 
                          [1, 2, 5], 
                          [1, 2, 3, 5], 
                          [1, 5]])

print(generarC1(transacciones))

[frozenset({1}), frozenset({2}), frozenset({3}), frozenset({4}), frozenset({5})]


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  after removing the cwd from sys.path.


In [9]:
for x in generarC1(transacciones):
    print(x)

frozenset({1})
frozenset({2})
frozenset({3})
frozenset({4})
frozenset({5})


Todos los conjuntos de tamaño 1 que superan el umbral $minS$ se combinan para crear conjuntos de tamaño $2$, los cuales también son testeados en busca de aquellos que superan también este umbral, $minS$.

En general, todos aquellos conjuntos de la $i$-ésima iteración que superan el umbral de soporte $minS$ (conjuntos de tamaño $|X|=i$), en la siguiente iteración ($i+1$) del algoritmo, se combinan entre ellos para generar nuevos conjuntos de tamaño $i+1$. 

La siguiente función se usa para identificar los conjuntos que superan el umbral de soporte:

In [10]:
def filtraPorSoporte(S, Ck, minS):
    conteo = {}
    for tr in S:
        for itemset in Ck:
            if itemset.issubset(tr):
                if itemset not in conteo: 
                    conteo[itemset] = 1
                else: 
                    conteo[itemset] += 1
    numItems = float(len(S))
    Ck_minS = []
    soporteCk = {}
    for itemset in conteo:
        soporte = conteo[itemset]/numItems### TU CODIGO AQUI ###
        if soporte >= minS:
            Ck_minS.insert(0, itemset)
        soporteCk[itemset] = soporte
    return Ck_minS, soporteCk


Podemos hacer el cálculo para obtener los conjuntos de tamaño 1 que ocurren en al menos el $50\%$ de las transacciones del conjunto de entrenamiento:


In [12]:
S = list(map(set,transacciones))
C1 = generarC1(transacciones)
print(S)

L1, soporteC1 = filtraPorSoporte(S,C1,0.5)
print(L1)
print(soporteC1)

# INICIO - NO TOCAR : necesario para la evaluación del notebook
answers['1'] = {'L1': L1,
                'soporteC1': soporteC1}
# FIN - NO TOCAR : necesario para la evaluación del notebook

[{2, 3, 4}, {1, 2, 5}, {1, 2, 3, 5}, {1, 5}]
[frozenset({5}), frozenset({1}), frozenset({3}), frozenset({2})]
{frozenset({2}): 0.75, frozenset({3}): 0.5, frozenset({4}): 0.25, frozenset({1}): 0.75, frozenset({5}): 0.75}



Se puede ver que el ítem 4 sólo aparece en la primera transacción, por lo que no superó el umbral de soporte fijado.

Probablemente la parte más sensible de este algoritmo consiste en generar los candidados (conjuntos de ítems) de una nueva iteración ($C_k$) dados los conjuntos frecuentes de la previa ($L_{k-1}$). 

Por ejemplo, dados los conjuntos frecuentes de la primera etapa $\{1\}$, $\{2\}$, $\{3\}$ y $\{5\}$, los candidados de tamaño 2 ($C_2$) serán:
$\{1,2\}$, $\{1,3\}$, $\{1,5\}$, $\{2,3\}$, $\{2,5\}$ y $\{3,5\}$.

En concreto, se podría hacer de la siguiente manera:


In [14]:
def generarCk(Lk1, k):
    Ck = []
    lenLk1 = len(Lk1)
    for i in range(lenLk1):
        for j in range(i+1, lenLk1): 
            L1 = list(Lk1[i])[:k-2]
            L1.sort()
            L2 = list(Lk1[j])[:k-2]
            L2.sort()
            if L1 == L2: # Si los primeros k-2 elementos son los mismos
                Ck.append(Lk1[i] | Lk1[j]) # hacemos la union de ambos conjuntos
    return Ck


Se puede comprobar que el resultado cuadra con lo esperado:


In [15]:
print('Conjuntos candidatos de tamaño 2,\nC2 =', generarCk(L1,2))

Conjuntos candidatos de tamaño 2,
C2 = [frozenset({1, 5}), frozenset({3, 5}), frozenset({2, 5}), frozenset({1, 3}), frozenset({1, 2}), frozenset({2, 3})]



Por último, sólo faltaría definir la función principal del algoritmo que itera entre la formación de conjuntos candidatos y el filtrado de aquellos que cumplen los requisitos de soporte mínimo:


In [16]:
def apriori(transacciones, minS = 0.5):
    S = list(map(set, transacciones))
    C1 = generarC1(transacciones)
    L1, soporteItemSets = filtraPorSoporte(S, C1, minS)
    L = [L1]
    k = 2
    while (len(L[k-2]) > 0):
        Ck = generarCk(L[k-2], k)
        Lk, soporteCk = filtraPorSoporte(S, Ck, minS)
        soporteItemSets.update(soporteCk)
        L.append(Lk)
        k += 1
    return L, soporteItemSets


Podemos finalmente buscar todos los conjuntos frecuentes (en este caso, soporte mínimo de $50\%$) de $S$ haciendo la siguiente llamada;

In [17]:
L, soporteCk = apriori(transacciones)

print('Conjuntos frecuentes de tamaño 1:',L[0])
print('Conjuntos frecuentes de tamaño 2:',L[1])
print('Conjuntos frecuentes de tamaño 3:',L[2])
print('Conjuntos frecuentes de tamaño 4:',L[3])

Conjuntos frecuentes de tamaño 1: [frozenset({5}), frozenset({1}), frozenset({3}), frozenset({2})]
Conjuntos frecuentes de tamaño 2: [frozenset({1, 2}), frozenset({2, 5}), frozenset({1, 5}), frozenset({2, 3})]
Conjuntos frecuentes de tamaño 3: [frozenset({1, 2, 5})]
Conjuntos frecuentes de tamaño 4: []



<hr /> 

<center><h1>Obtener reglas de asociación a partir de conjuntos frecuentes</h1></center>

En teoría hemos visto cómo encontrar una serie de conjuntos frecuentes de ítems. Dado un conjunto frecuente, se puede generar una regla de asociación de la siguiente manera.

Se trata de recorrer todo el listado de conjuntos frecuentes y estudiar la conveniencia de nuevas reglas dado un valor mínimo de confianza:


In [18]:
def generarReglas(L, soporteItemSets, minC=0.7):
    lReglas = []
    # para crear reglas, sólo podemos considerar conjuntos de tamaño 2 o mayor
    if (len(L) == 1):
        return lReglas
    for i in range(1, len(L)): 
        for itemset in L[i]:
            H1 = [frozenset([item]) for item in itemset]
            nuevasReglas = reglasConfianzaMinima(itemset, H1, soporteItemSets, minC)
            if (len(nuevasReglas) > 0):
                lReglas = lReglas+nuevasReglas
    return lReglas


Dado un conjunto frecuente $A$ de ítems específico (de tamaño $>2$), recorreremos todos los elementos $e\in A$ y consideraremos la conveniencia de cada regla $A\backslash e \to e$, para lo que calcularemos el valor de confianza de la siguiente manera:

$$confianza(A\backslash e \to e;S)=\frac{soporte(A;S)}{soporte(A\backslash e;S)}$$

Así, la función puede definirse como:


In [20]:
def reglasConfianzaMinima(itemset, H, soporteItemSets, minC=0.7):
    lReglas = []
    for consecuente in H:
        # Calcular confianza
        itemsetSINcons = itemset-consecuente
        conf = soporteItemSets[itemset]/len(itemsetSINcons)### TU CODIGO AQUI ###
        if conf >= minC: 
            print(itemset-consecuente,'-->',consecuente,'con confianza:',conf)
            lReglas.append((itemset-consecuente, consecuente, conf))
    return lReglas


Finalmente, podemos buscar las reglas que tengan un mínimo de confianza del $70\%$:


In [21]:
reglas = generarReglas(L, soporteCk, minC=0.7)

print('\nLa confianza de la primera regla es:',reglas[0][2])

# INICIO - NO TOCAR : necesario para la evaluación del notebook
answers['2'] = {'reglas': L1}
# FIN - NO TOCAR : necesario para la evaluación del notebook

frozenset({5}) --> frozenset({1}) con confianza: 0.75
frozenset({1}) --> frozenset({5}) con confianza: 0.75

La confianza de la primera regla es: 0.75



Es curioso el hecho de que girando la regla $5 \to 1$ obtenemos otra regla con la confianza requerida y, sin embargo, al hacer lo mismo con la regla $3\to 2$, la regla resultante no supera el umbral marcado.


<hr />

<center><h1>Librerias de Python</h1></center>

Una librería interesante que incluye el algoritmo Apriori es <b>apyori</b> (hay que descargarlo con `!wget https://raw.githubusercontent.com/ymoch/apyori/master/apyori.py`). Veamos como funciona.

Para empezar, cargamos las librerías necesarias y el conjunto de transacciones que usaremos:


In [22]:
!wget https://raw.githubusercontent.com/ymoch/apyori/master/apyori.py
!wget https://raw.githubusercontent.com/flifuehu/viu-unsupervised-learning/master/datasets/apriori/store_data.csv

--2021-04-03 18:42:42--  https://raw.githubusercontent.com/ymoch/apyori/master/apyori.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 14558 (14K) [text/plain]
Saving to: ‘apyori.py’

apyori.py           100%[===================>]  14.22K  --.-KB/s    in 0s      

2021-04-03 18:42:43 (76.7 MB/s) - ‘apyori.py’ saved [14558/14558]

--2021-04-03 18:42:43--  https://raw.githubusercontent.com/flifuehu/viu-unsupervised-learning/master/datasets/apriori/store_data.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.108.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 302908 (296K) [text/p

In [24]:
import pandas as pd  
from apyori import apriori  

matriz_datos = pd.read_csv('store_data.csv', header=None)


El algoritmo necesita que las transacciones se le pasen como una lista de listas, por lo que el primer paso es transformar la matriz de datos anterior en una lista de ese estilo:


In [25]:
transacciones = []  
for i in np.arange(matriz_datos.shape[0]):  
    transacciones.append([str(matriz_datos.values[i,j]) 
                          for j in np.arange(matriz_datos.shape[1])
                          if str(matriz_datos.values[i,j]) != 'nan'])


Podemos inspeccionar unas pocas transacciones para ver cómo lucen:


In [26]:
for i in np.arange(8):
    print(' + Transacción',i+1,':',', '.join(transacciones[i]))

 + Transacción 1 : shrimp, almonds, avocado, vegetables mix, green grapes, whole weat flour, yams, cottage cheese, energy drink, tomato juice, low fat yogurt, green tea, honey, salad, mineral water, salmon, antioxydant juice, frozen smoothie, spinach, olive oil
 + Transacción 2 : burgers, meatballs, eggs
 + Transacción 3 : chutney
 + Transacción 4 : turkey, avocado
 + Transacción 5 : mineral water, milk, energy bar, whole wheat rice, green tea
 + Transacción 6 : low fat yogurt
 + Transacción 7 : whole wheat pasta, french fries
 + Transacción 8 : soup, light cream, shallot



Con estas transacciones, ya podemos aplicar el algoritmo Apriori dados unos requisitos de soporte y confianza mínimas:


In [27]:
lReglas = apriori(transacciones, min_support=0.005, min_confidence=0.5)


En este caso, también podríamos fijar la métrica <i>Lift</i>.

Podemos observar las reglas resultantes:


In [28]:
lReglas = list(lReglas) 
print('Se han encontrado',len(lReglas),'reglas con los requisitos impuestos')
print('La primera regla es:\n',lReglas[0])

Se han encontrado 20 reglas con los requisitos impuestos
La primera regla es:
 RelationRecord(items=frozenset({'chocolate', 'mineral water', 'chicken'}), support=0.007598986801759766, ordered_statistics=[OrderedStatistic(items_base=frozenset({'chocolate', 'chicken'}), items_add=frozenset({'mineral water'}), confidence=0.5181818181818182, lift=2.1738712629652226)])



Podemos ver que tenemos muchísima información sobre la regla codificada en las respuestas.

Si queremos hacer un recorrido por todas ellas, podemos hacerlo de la siguiente manera:


In [29]:
for item in lReglas:

    par = [x for x in item[0]]
    print("Regla: " + par[0] + " -> " + par[1])

    print(" + Soporte:   {0:1.3f}".format(item[1]))
    print(" + Confianza: {0:1.3f}".format(item[2][0][2]))
    print(" + Lift:      {0:1.3f}".format(item[2][0][3]))
    print()

Regla: chocolate -> mineral water
 + Soporte:   0.008
 + Confianza: 0.518
 + Lift:      2.174

Regla: chocolate -> mineral water
 + Soporte:   0.008
 + Confianza: 0.504
 + Lift:      2.115

Regla: soup -> chocolate
 + Soporte:   0.006
 + Confianza: 0.553
 + Lift:      2.318

Regla: cooking oil -> eggs
 + Soporte:   0.006
 + Confianza: 0.545
 + Lift:      2.288

Regla: ground beef -> eggs
 + Soporte:   0.010
 + Confianza: 0.507
 + Lift:      2.126

Regla: ground beef -> frozen vegetables
 + Soporte:   0.009
 + Confianza: 0.543
 + Lift:      2.279

Regla: ground beef -> frozen vegetables
 + Soporte:   0.009
 + Confianza: 0.512
 + Lift:      2.940

Regla: frozen vegetables -> mineral water
 + Soporte:   0.007
 + Confianza: 0.576
 + Lift:      2.418

Regla: soup -> frozen vegetables
 + Soporte:   0.005
 + Confianza: 0.633
 + Lift:      2.657

Regla: frozen vegetables -> spaghetti
 + Soporte:   0.006
 + Confianza: 0.506
 + Lift:      2.906

Regla: ground beef -> milk
 + Soporte:   0.011
 + 

# RECUERDA EJECUTAR ESTA CELDA ÚNICAMENTE CUANDO HAYAS FINALIZADO EL NOTEBOOK

In [30]:
################################################################################
# ATENCIÓN, MUY IMPORTANTE: EJECUTA ESTA CELDA SIN MODIFICAR NADA!
#
# ESTA CELDA SE ENCARGA DE COMPRIMIR LOS RESULTADOS OBTENIDOS Y MANDARLOS
# AL SERVIDOR PARA SU VERIFICACIÓN Y ALMACENAJE.
#
# SI NO LA EJECUTAS, ¡¡¡NO PODRÁS SER EVALUADX!!!
################################################################################

import zmq
import base64
import pickle
import bz2
import sys

def send_results(answers):

    def compress_data(data):
        data_pkl = pickle.dumps(data)
        data_bz2 = bz2.compress(data_pkl)
        return data_bz2

    if NOMBRE == 'TuNombre' or APELLIDOS == 'TusApellidos' or \
    NOMBRE.strip() == '' or APELLIDOS.strip() == '':
        print('Rellena tu nombre y apellidos en la primera celda del notebook!')
    else:
        print('Conectando con el servidor de evaluación...', end='\t')
        context = zmq.Context()
        socket = context.socket(zmq.REQ)
        socket.connect(base64.b64decode(b'dGNwOi8vMTU4LjQyLjE3MC4xMzU6MzM4OQ==').decode('ascii'))
        print('OK')

        # Compressing data
        print('Comprimiendo las respuestas...', end='\t')
        data = compress_data(answers)
        print('OK')

        print('Enviando datos...', end='\t')
        socket.send(data)
        message = socket.recv()
        if message == data:
            print(f'OK')
            print('Entrega realizada con éxito! :)')
        else:
            print('ERROR! Por favor vuelve a ejecutar la celda. Si los problemas persisten ponte en contacto con el profesor.')    


send_results(answers)

Conectando con el servidor de evaluación...	OK
Comprimiendo las respuestas...	OK
Enviando datos...	OK
Entrega realizada con éxito! :)
